### TESTING 

In [333]:
import pandas as pd
from sklearn.ensemble import IsolationForest

In [358]:
df = pd.read_csv('all_combined_EMA2233M10296.csv')
df = df.dropna(subset='cycle_time')
# df = df.rename(columns={'CT': 'cycle_time', 'Shot Time': 'shot_time'})
df = df[['shot_time','cycle_time']]

In [359]:
data = ['cycle_time']

In [360]:
if_model = IsolationForest(contamination='auto',random_state=42)

In [361]:
if_model.fit(df[data])
df['anomoly_score'] = if_model.decision_function(df[data])
df['anomoly_score_inverse']=if_model.score_samples(df[data])
# df['anomoly_score'] = df['anomoly_score']+1
df['anomoly_score_inverse'] = df['anomoly_score_inverse']+1

In [351]:
df

,shot_time,cycle_time,anomoly_score,anomoly_score_inverse
0,2024-04-18 08:12:19,122.8,0.130316,0.630316
1,2024-04-18 08:14:22,128.5,0.005178,0.505178
2,2024-04-18 08:16:30,54.8,-0.239112,0.260888
3,2024-04-18 08:17:25,99.3,-0.142385,0.357615
4,2024-04-18 08:19:04,175.8,-0.018856,0.481144
...,...,...,...,...
1493,2024-04-22 05:20:32,124.2,0.173204,0.673204
1495,2024-04-22 05:22:36,124.2,0.173204,0.673204
1496,2024-04-22 05:24:40,124.3,0.171148,0.671148
1497,2024-04-22 05:26:45,124.2,0.173204,0.673204


In [362]:
df['output'] = if_model.predict(df[data])

In [353]:
df

,shot_time,cycle_time,anomoly_score,anomoly_score_inverse,output
0,2024-04-18 08:12:19,122.8,0.130316,0.630316,1
1,2024-04-18 08:14:22,128.5,0.005178,0.505178,1
2,2024-04-18 08:16:30,54.8,-0.239112,0.260888,-1
3,2024-04-18 08:17:25,99.3,-0.142385,0.357615,-1
4,2024-04-18 08:19:04,175.8,-0.018856,0.481144,-1
...,...,...,...,...,...
1493,2024-04-22 05:20:32,124.2,0.173204,0.673204,1
1495,2024-04-22 05:22:36,124.2,0.173204,0.673204,1
1496,2024-04-22 05:24:40,124.3,0.171148,0.671148,1
1497,2024-04-22 05:26:45,124.2,0.173204,0.673204,1


In [363]:
# Define a function to apply to each row
def assign_anomaly_score(row):
    if row['anomoly_score_inverse']<  0.3:
        return -1
    else:
        return 1

# Apply the function to create the new column
df['output_anomaly_score'] = df.apply(assign_anomaly_score, axis=1)

In [365]:
df.to_csv('result/all_combined_EMA2324M11078.csv')

In [259]:
# df['gradient_fwd'] = df['cycle_time'].diff(1)
# df['gradient_bck'] = df['cycle_time'].diff(-1)

In [364]:
df['gradient_bck'] = df['cycle_time'].diff(-1) # This will fill the first value with zero, because the difference is x = x[i] - x[i+1]
df['gradient_bck'] = df['gradient_bck'].abs()
df['gradient_fwd'] = df['cycle_time'].diff()
df['gradient_fwd'] = df['gradient_fwd'].abs()
# try:
#     df['gradient_fwd'].iloc[0] = df['gradient_fwd'].iloc[1]
# except IndexError:
#     pass
# try:
#     df['gradient_bck'].iloc[-1] = df['gradient_bck'].iloc[-2]
# except IndexError:
#     pass 
# n_consecutive_ones = 2  
# df['Relabel'] = check_consecutive_ones(df['Proc Change CT'], n_consecutive_ones).astype(int)
tolerance = 5
# df.loc[(df['Relabel'] == 1) & (df['Gradient'] <= tolerance), 'Proc Change CT'] = 0
df.loc[ (df['gradient_fwd'] >= tolerance) & (df['gradient_bck'] >= tolerance), 'output_anomaly_score'] = -1
# df.loc[ (df['gradient_fwd'] >= tolerance), 'output'] = -1
df

,shot_time,cycle_time,anomoly_score,anomoly_score_inverse,output,output_anomaly_score,gradient_bck,gradient_fwd
0,2022-05-10 19:23:23,68.3,-0.032356,0.467644,-1,1,11.4,NaN
1,2022-05-10 19:24:31,56.9,-0.062461,0.437539,-1,1,0.3,11.4
2,2022-05-10 19:25:28,56.6,-0.094157,0.405843,-1,1,0.1,0.3
3,2022-05-10 19:26:25,56.5,-0.096987,0.403013,-1,1,0.0,0.1
4,2022-05-10 19:27:21,56.5,-0.096987,0.403013,-1,1,0.0,0.0
...,...,...,...,...,...,...,...,...
53366,2024-02-26 23:57:13,59.5,0.134697,0.634697,1,1,0.1,0.0
53368,2024-02-26 23:58:13,59.6,0.131312,0.631312,1,1,0.2,0.1
53369,2024-02-26 23:59:12,59.4,0.131212,0.631212,1,1,0.0,0.2
53370,2024-02-27 00:00:12,59.4,0.131212,0.631212,1,1,940.5,0.0


### LABELLING PRODUCTION SEAESON WISE

In [183]:
import numpy as np
import math
def convert_strtime_to_datetime(data):
    """
    Converts the 'TFF' column in the given DataFrame to datetime format.

    Parameters:
    data (DataFrame): The input DataFrame containing the 'TFF' column.

    Returns:
    DataFrame: The modified DataFrame with the 'TFF' column converted to datetime format.
    """
    data["TFF"] = pd.to_datetime(data["TFF"], format="%Y%m%d%H%M%S")
    return data


def get_time_difference(data):
    """
    Calculates the time difference between consecutive rows in the 'TFF' column of the given DataFrame.

    Parameters:
    data (DataFrame): The input DataFrame containing the 'TFF' column.

    Returns:
    Series: The time differences between consecutive rows in hours.
    """
    return (data["TFF"] - data["TFF"].shift(1)).dt.total_seconds() / 3600


def get_list_of_prod_seasons(data):
    """
    Splits the given DataFrame into multiple sections based on time differences in the 'TFF' column.

    Parameters:
    data (DataFrame): The input DataFrame containing the 'TFF' column.

    Returns:
    list: A list of DataFrames, where each DataFrame represents a section of consecutive rows with time differences greater than or equal to 4 hours.
    """
    data = data.reset_index()
    data = convert_strtime_to_datetime(data)

    data["time_difference"] = get_time_difference(data)

    break_points = data[
        data["time_difference"] >= 4
    ].index.values
    if break_points.shape[0] > 0:
        list_of_sections_indices = np.sort(
            np.insert(break_points, len(break_points), data.index.values[-1] + 1)
        )
        ps = [
            data.iloc[:x] if i == 0 else data.iloc[list_of_sections_indices[i - 1] : x]
            for i, x in enumerate(list_of_sections_indices)
        ]
    else:
        ps = [data]
    return ps
def assign_anomaly_score(row):
    if row['anomoly_score_inverse']<  0.3:
        return -1
    else:
        return 1
    
def convert_to_desired_format(date_str):
    # Convert string to datetime object
    date_obj = pd.to_datetime(date_str)
    # Convert datetime object to string with the desired format
    formatted_date = date_obj.strftime("%Y%m%d%H%M%S")
    return formatted_date

In [184]:
df = pd.read_csv('all_combined_EMA2233M10296.csv')
df = df.dropna(subset='cycle_time')
df['cycle_time'] = df['cycle_time'].apply(math.floor)
df = df[['shot_time','cycle_time']]
df['TFF'] = df['shot_time'].apply(lambda x: convert_to_desired_format(x))
final_df = pd.DataFrame()
# Get list of production seasons
prod_seasons = get_list_of_prod_seasons(df)

# Process data for each production season
for ps_data in prod_seasons:
    if_model = IsolationForest(contamination='auto',random_state=42)
    if_model.fit(ps_data[['cycle_time']])
    # Calculate anomaly scores
    ps_data['anomaly_score'] = if_model.decision_function(ps_data[['cycle_time']])
    ps_data['anomoly_score_inverse'] = if_model.score_samples(ps_data[['cycle_time']]) + 1
    ps_data['output'] = if_model.predict(ps_data[['cycle_time']])
    ps_data['output_anomaly_score'] = ps_data.apply(assign_anomaly_score, axis=1)
    final_df = pd.concat([final_df, pd.DataFrame(ps_data)], ignore_index=True)
final_df= final_df[['shot_time','cycle_time','anomaly_score','anomoly_score_inverse','output_anomaly_score','output']]
final_df.to_csv('result/labelled_all_combined_EMA2233M10296.csv')
final_df

,shot_time,cycle_time,anomaly_score,anomoly_score_inverse,output_anomaly_score,output
0,2022-05-10 19:23:23,68,-0.285656,0.214344,-1,-1
1,2022-05-10 19:24:31,56,0.100849,0.600849,1,1
2,2022-05-10 19:25:28,56,0.100849,0.600849,1,1
3,2022-05-10 19:26:25,56,0.100849,0.600849,1,1
4,2022-05-10 19:27:21,56,0.100849,0.600849,1,1
...,...,...,...,...,...,...
49657,2024-02-26 23:57:13,59,0.076946,0.576946,1,1
49658,2024-02-26 23:58:13,59,0.076946,0.576946,1,1
49659,2024-02-26 23:59:12,59,0.076946,0.576946,1,1
49660,2024-02-27 00:00:12,59,0.076946,0.576946,1,1


### PLOTS

In [317]:
import plotly.graph_objects as go
import os

def process_file(file_path, output_directory):
    data = pd.read_csv(file_path)
    data['shot_time'] = pd.to_datetime(data['shot_time'])
    data = data.dropna(subset=['cycle_time'])

    # Set color for markers based on output_anomaly_score column
    colors = ['red' if score == -1 else 'rgb(31, 119, 180)' for score in data['output']]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['shot_time'], y=data['cycle_time'], 
                             mode='markers+lines', line=dict(color='darkgrey', width=1), 
                             marker=dict(symbol='circle', size=8, color=colors)))

    fig.update_layout(
        title='Cycle Time vs Measurement Date ',
        xaxis_title='Measurement Date',
        yaxis_title='Cycle Time (seconds)',
        template='plotly_white',
        plot_bgcolor='lightgrey',
        xaxis=dict(
            showgrid=True,
            gridcolor='white',
            tickfont=dict(size=14) 
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='white',
            tickfont=dict(size=14)
        )
    )

    # Assuming file_path and output_directory are defined
    filename = os.path.join(output_directory, '{}_ct.html'.format(os.path.basename(file_path).split('.')[0]))
    fig.write_html(filename)

output_directory = "ct_plots"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

input_directory = "result"
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_directory, filename)
        process_file(file_path, output_directory)


In [366]:
import pandas as pd
import plotly.graph_objects as go
import os

def process_file(file_path, output_directory):
    data = pd.read_csv(file_path)
    data['shot_time'] = pd.to_datetime(data['shot_time'])
    data = data.dropna(subset=['cycle_time'])

    # Set color for markers based on output_anomaly_score column
    colors = ['red' if score == -1 else 'rgb(31, 119, 180)' for score in data['output_anomaly_score']]

    # Add hover text with anomoly_score_inverse
    hover_text = [f"Cycle Time: {ct}<br>Anomaly Score Inverse: {score_inv}" 
                  for ct, score_inv in zip(data['cycle_time'], data['anomoly_score_inverse'])]

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['shot_time'], y=data['cycle_time'], 
                             mode='markers+lines', line=dict(color='darkgrey', width=1), 
                             marker=dict(symbol='circle', size=8, color=colors),
                             hoverinfo='text',
                             hovertext=hover_text))

    fig.update_layout(
        title='Cycle Time vs Measurement Date ',
        xaxis_title='Measurement Date',
        yaxis_title='Cycle Time (seconds)',
        template='plotly_white',
        plot_bgcolor='lightgrey',
        xaxis=dict(
            showgrid=True,
            gridcolor='white',
            tickfont=dict(size=14) 
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='white',
            tickfont=dict(size=14)
        )
    )

    # Assuming file_path and output_directory are defined
    filename = os.path.join(output_directory, '{}_ct.html'.format(os.path.basename(file_path).split('.')[0]))
    fig.write_html(filename)

output_directory = "ct_plots"
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

input_directory = "result"
for filename in os.listdir(input_directory):
    if filename.endswith(".csv"):
        file_path = os.path.join(input_directory, filename)
        process_file(file_path, output_directory)
